<a href="https://colab.research.google.com/github/maxkleiner/maXbox/blob/master/nltk4sent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import string

from nltk.corpus import movie_reviews as mr
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('movie_reviews')
from nltk.stem import PorterStemmer
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


But the no. of unique words in a corpus can be very huge. We could restrict our model to extract features that are the most salient. But to do so we need to know what is the top N most frequent words in the corpus. 


In [3]:
# nltk.download('stopwords')
# nltk.download('movie_reviews')
stop = stopwords.words('english') + list(string.punctuation)
# This will extract key-value pairs, 
# where the keys are words and values are its count in the corpus.
vocabulary = FreqDist(w.lower() for w in mr.words() if w.lower() not in stop)
print(vocabulary.most_common(10))
#print(list(vocabulary.keys())[:10])
# the least uses words
print(vocabulary.most_common()[-10:])

#word_features = FreqDist(chain(*[i for i,j in documents]))
word_features = list(vocabulary.keys())[:10]
print(word_features)

# vocabulary[1:50]
#vocabulary

documents = [([w for w in mr.words(i) if w.lower() not in stop], # Words in document.
               i.split('/')[0]) # Tag.
                for i in mr.fileids()]
print(documents[0][:10]) # First document

[('film', 9517), ('one', 5852), ('movie', 5771), ('like', 3690), ('even', 2565), ('good', 2411), ('time', 2411), ('story', 2169), ('would', 2109), ('much', 2049)]
[('paneled', 1), ('vainly', 1), ('snoots', 1), ('obstructions', 1), ('obscuring', 1), ('tangerine', 1), ('timbre', 1), ('powaqqatsi', 1), ('keyboardist', 1), ('capitalized', 1)]
['plot', 'two', 'teen', 'couples', 'go', 'church', 'party', 'drink', 'drive', 'get']
(['plot', 'two', 'teen', 'couples', 'go', 'church', 'party', 'drink', 'drive', 'get', 'accident', 'one', 'guys', 'dies', 'girlfriend', 'continues', 'see', 'life', 'nightmares', 'deal', 'watch', 'movie', 'sorta', 'find', 'critique', 'mind', 'fuck', 'movie', 'teen', 'generation', 'touches', 'cool', 'idea', 'presents', 'bad', 'package', 'makes', 'review', 'even', 'harder', 'one', 'write', 'since', 'generally', 'applaud', 'films', 'attempt', 'break', 'mold', 'mess', 'head', 'lost', 'highway', 'memento', 'good', 'bad', 'ways', 'making', 'types', 'films', 'folks', 'snag', '

Classify the sentiment from NLTK to Scikit


In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm, datasets
from sklearn.metrics import accuracy_score

#X = mr.words
#y = mr.tag

An NLTK's CategorizedPlaintextCorpusReader object isn't a dtype for pandas.

That being said, you can convert the movie reviews into list of tuples and then populate a dataframe as such:

In [5]:
import pandas as pd
from nltk.corpus import movie_reviews as mr

reviews = []
for fileid in mr.fileids():
    tag, filename = fileid.split('/')
    reviews.append((filename, tag, mr.raw(fileid)))

df = pd.DataFrame(reviews, columns=['filename', 'tag', 'text'])

df.head(7)

,filename,tag,text
0,cv000_29416.txt,neg,"plot : two teen couples go to a church party ,..."
1,cv001_19502.txt,neg,the happy bastard's quick movie review \ndamn ...
2,cv002_17424.txt,neg,it is movies like these that make a jaded movi...
3,cv003_12683.txt,neg,""" quest for camelot "" is warner bros . ' firs..."
4,cv004_12641.txt,neg,synopsis : a mentally unstable man undergoing ...
5,cv005_29357.txt,neg,capsule : in 2176 on the planet mars police ta...
6,cv006_17022.txt,neg,"so ask yourself what "" 8mm "" ( "" eight millime..."


We are now ready to classify
The purpose is to use this data for sentiment analysis. while converting the data using pandas, glorious dataframe


In [15]:
nltk.download('punkt')
stoplist = stopwords.words('english') + list(string.punctuation)
def preprocess(atext):
    return [ps.stem(word) for word in nltk.word_tokenize(atext) \
                 if word.lower() not in stoplist and not word.isdigit()]

# it doesnt apply the preprocess to the text vocab, shape remains the same?!
# df['text'].apply(preprocess) 

X = df['text']
y = df['tag']
X= X.map(lambda x:' '.join([ps.stem(word) for word in x.split(' ') \
                   # if not word in stoplist and not word.isdigit()]))
                     if word.lower() not in stoplist and not word.isdigit()]))
print('neg:',sum(df['tag']=='neg'), 'pos:',sum(df['tag']=='pos'))
print('single word count ',sum(df.text.str.count('film')))
print('preproc: ','\n',X[0:5])

# df= pd.read_csv('../input/movie-review/movie_review.csv') no need cause of df

vect = CountVectorizer(ngram_range=(1, 2), min_df=3)
#vect = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', min_df=4, max_df=0.8) 
#, preprocessor=preprocess)


Xv = vect.fit_transform(X)
print('shape',Xv.shape)

# Whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.
X_train, X_test, y_train, y_test = \
         train_test_split(Xv, y, test_size=0.3, random_state=42, stratify=None, shuffle=True)


# model = BernoulliNB(alpha=1.0)
model = svm.LinearSVC(random_state=0, C=0.01) #, gamma='scale')

model.fit(X_train, y_train)

p_train = model.predict(X_train)
p_test = model.predict(X_test)

acc_train = accuracy_score(y_train, p_train)
acc_test = accuracy_score(y_test, p_test)

print(f'Train ACC: {acc_train}, Test ACC: {acc_test}')

from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_train, p_train, labels=['neg', 'pos']),'\n')
print(confusion_matrix(y_test, p_test, labels=['neg', 'pos']),'\n')

print(df.info())



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
neg: 1000 pos: 1000
single word count  11607
preproc:  
 0    plot two teen coupl go church parti drink driv...
1    happi bastard' quick movi review \ndamn y2k bu...
2    movi like make jade movi viewer thank invent t...
3     quest camelot warner bro first feature-length...
4    synopsi mental unstabl man undergo psychothera...
Name: text, dtype: object
shape (2000, 46184)
Train ACC: 1.0, Test ACC: 0.835
[[698   0]
 [  0 702]] 

[[255  47]
 [ 52 246]] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2000 non-null   object
 1   tag       2000 non-null   object
 2   text      2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB
None


In [0]:
from pandas.api.types import is_categorical_dtype
#import shap
def cat_to_code(df_orig):
    """Convert dataframe with categoricals to numerical array"""
    df_mod = df_orig.copy()
    for col in df_orig.columns:
        if is_categorical_dtype(df_orig[col]):
            df_mod[col] = df_orig[col].cat.codes
    X = df_mod.values
    return X

#x_codes = cat_to_code(X)

#exp = shap.TreeExplainer(model=model, data=x_codes)
#print(exp.shap_values(X))

In the end we check the distribution of negative and positive contribution words

In [0]:
from nltk.probability import FreqDist, ConditionalFreqDist
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()
stops = stopwords.words('english') + list(string.punctuation)

#polar = [w.lower() for w in mr.words() if w.lower() not in stops]

def best_word_feats(words):
    return dict([(word, True) for word in words if word not in stops])

all_words = [word.lower() for word in mr.words()]
# print first 10 words
print (all_words[:10])

all_words_clean = []
for word in all_words:
    if word not in stops:
        all_words_clean.append(word)
 
print (all_words_clean[:10])
print ('distinctive words: ',len(set(all_words_clean)))

all_words_freq = FreqDist(all_words_clean)
print (all_words_freq)

# for word in best_word_feats(mr.words(categories=['neg'])):
for word in mr.words(categories=['neg']):
    #word_fd.inc(word.lower())
    all_words_freq[word.lower()] +=1 
    label_word_fd['neg'][word.lower()] +=1
print(all_words_freq.most_common()[:20])
print(all_words_freq.most_common()[-20:])
 
for word in mr.words(categories=['pos']):
    all_words_freq[word.lower()] += 1
    label_word_fd['pos'][word.lower()] += 1  
print(label_word_fd['pos'].most_common()[:20])
print(label_word_fd['pos'].most_common()[-20:]) 

neg_word_count = label_word_fd['neg'].N()
pos_word_count = label_word_fd['pos'].N()

# print 10 most frequently occurring words
print (all_words_freq.most_common(20))

print(neg_word_count, pos_word_count)

from matplotlib import pyplot as plt

def feat_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

#feat_importances(model.coef_, features_names)
#importances = model.feature_importances_
print(vect.get_feature_names()[0:15])
# pd.Series(abs(model.coef_[0]), index=vect.get_feature_names).nlargest(10).plot(kind='barh')
 